In [1]:
!pip install imblearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
# sklearn packages (for pre-processing, model-building, evaluators, cross-validators, etc.)
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report, roc_auc_score, f1_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder

# Imblearn packages (for over_sampling and imblearn pipelines)
from imblearn import over_sampling
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from imblearn.pipeline import Pipeline, make_pipeline  ## Using packages from imblearn.pipeline for making pipelines

## xgboost packages
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

!pip install tqdm
from tqdm import tqdm

from random import sample

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 4.0 MB/s eta 0:00:00a 0:00:01


In [52]:
# create 10 data sets according to time, proportion of frauds, and (cluster?)
df_cct = pd.read_parquet("/home/onyxia/work/df_cct_stand2.parquet")
df_cct

,User,Card,Month,Day,Hours,year_diff,Amount,Use Chip,Card Brand,Card Type,...,nb_daily_declines_card,nb_monthly_declines_card,bad_pin,insufficient_balance,hr_nbt/last_30d_av_hr_nbt,day_nbt/last_30d_av_day_nbt,last_3d_amt/nbt,label_global,Is Fraud?,pos_amount
0,0,0,0.333333,-1.000000,0.000000,-1.0,0.693441,Swipe Transaction,Visa,Debit,...,0.0,-0.5,0,0,0.430959,0.131732,1.006352,0,0,1
1,0,0,0.333333,-1.000000,0.000000,-1.0,-0.394334,Swipe Transaction,Visa,Debit,...,0.0,-0.5,0,0,0.430959,0.131732,0.011261,0,0,1
2,0,0,0.333333,-0.933333,0.000000,-1.0,0.537004,Swipe Transaction,Visa,Debit,...,0.0,-0.5,0,0,-0.430959,0.131732,0.247553,0,0,1
3,0,0,0.333333,-0.933333,1.571429,-1.0,0.634962,Swipe Transaction,Visa,Debit,...,0.0,-0.5,0,0,-0.287306,0.131732,0.410505,0,0,1
4,0,0,0.333333,-0.866667,0.000000,-1.0,0.359179,Swipe Transaction,Visa,Debit,...,0.0,-0.5,0,0,-0.184697,0.131732,0.407362,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,-0.833333,0.800000,0.916667,1.0,0.059421,Chip Transaction,Visa,Credit,...,0.0,0.0,0,0,2.665306,1.144988,0.221986,0,0,0
24386896,1999,1,-0.833333,0.800000,0.916667,1.0,0.059421,Chip Transaction,Visa,Credit,...,0.0,0.0,0,0,2.622500,1.131036,0.198821,0,0,1
24386897,1999,1,-0.833333,0.866667,-0.333333,1.0,0.260751,Chip Transaction,Visa,Credit,...,0.0,0.0,0,0,0.099149,-0.722775,-0.256704,0,0,1
24386898,1999,1,-0.833333,0.866667,0.750000,1.0,-0.365911,Chip Transaction,Visa,Credit,...,0.0,0.0,0,0,0.096847,-0.730202,-0.288962,0,0,1


In [12]:
df = pd.read_parquet("/home/onyxia/work/cct.parquet")

In [ ]:
df_amount = pd.read_parquet("/home/onyxia/work/df_all.parquet")

In [15]:
df.sort_values(["User","Year","Month","Day","Time"], inplace=True)

In [17]:
df.reset_index(drop=True, inplace=True)

In [25]:
df.sort_values(["Year","Month", "Day","Time"], inplace=True)

In [33]:
df.Year.unique()

array([1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [53]:
df_cct["abs_amount"] = list(df_amount.loc[df_cct.index]["Amount"].abs().values)

In [73]:
df_cct  = df_cct.loc[df.index]
df_cct.reset_index(drop=True,inplace=True)

In [78]:
df_cct.to_parquet("/home/onyxia/work/df_manual_sampling.parquet")

In [88]:
# For "Use Chip":

enc = OneHotEncoder(handle_unknown='ignore')

features_array = enc.fit_transform(df_cct[["Use Chip"]]).toarray()

features_labels = np.hstack(np.array(enc.categories_))

df_cct = pd.concat([df_cct, pd.DataFrame(features_array, columns = features_labels)], axis = 1)

In [89]:
# For "Card Brand":

enc = OneHotEncoder(handle_unknown='ignore')

features_array = enc.fit_transform(df_cct[["Card Brand"]]).toarray()

features_labels = np.hstack(np.array(enc.categories_))

df_cct = pd.concat([df_cct, pd.DataFrame(features_array, columns = features_labels)], axis = 1)

In [90]:
# For "Card Type":

enc = OneHotEncoder(handle_unknown='ignore')

features_array = enc.fit_transform(df_cct[["Card Type"]]).toarray()

features_labels = np.hstack(np.array(enc.categories_))

df_cct = pd.concat([df_cct, pd.DataFrame(features_array, columns = features_labels)], axis = 1)

In [91]:
# For "label global":

enc = OneHotEncoder(handle_unknown='ignore')

features_array = enc.fit_transform(df_cct[["label_global"]]).toarray()

features_labels = np.hstack(np.array(enc.categories_))

df_cct = pd.concat([df_cct, pd.DataFrame(features_array, columns = features_labels)], axis = 1)

In [92]:
df_cct.rename(columns={0:"0", -1: "-1", 1:"1", 2:"2", 3: "3", 4:"4", 5:"5", 6:"6"}, inplace=True)


In [97]:
# selecting features
# dropped card, nb_weekly_declines, day_of_week, nb_daily_declines, credit, debit, amex,  discover,
# mastercard, visa
features = ['Month', 'Day', 'Hours', 'year_diff', 'Amount', 'pos_amount',
       'Credit Limit', 'delta_t_s',
       'delta_t_s_card', 'amt/daily_income', 'Retired', 'daily_amount',
       'nb_monthly_declines_card', 'bad_pin',
       'insufficient_balance', 'hr_nbt/last_30d_av_hr_nbt',
       'day_nbt/last_30d_av_day_nbt', 'last_3d_amt/nbt', 'Chip Transaction', 'Online Transaction',
       'Swipe Transaction','Debit (Prepaid)',"Is Fraud?",
        "-1","0","1","2","3","4","5","6"]

df_selec = df_cct[features] # 21 features + clusters

### Manual Split

In [98]:
df_split = np.array_split(df_selec, 10)

In [99]:
for i in range(10):
    df_fraud = df_split[i][df_split[i]["Is Fraud?"] == 1]
    n_fraud = len(df_split[i][df_split[i]["Is Fraud?"] == 1])
    df_no_fraud = df_split[i][df_split[i]["Is Fraud?"] == 0]
    df_no_fraud_sampled = df_no_fraud.sample(int(0.95*(n_fraud/0.05)), random_state = 0)
    df_split[i] = pd.concat([df_no_fraud_sampled, df_fraud], axis = 0).sort_index()

In [100]:
for i in range(10):
    print(len(df_split[i]))

33940
63580
96880
78820
26660
61400
56440
80760
42300
54360


In [118]:
X = {}
y = {}
for i in range(10):
       X[i] = df_split[i][['Month', 'Day', 'Hours', 'year_diff', 'Amount', 'pos_amount',
              'Credit Limit', 'delta_t_s',
              'delta_t_s_card', 'amt/daily_income', 'Retired', 'daily_amount',
              'nb_monthly_declines_card', 'bad_pin',
              'insufficient_balance', 'hr_nbt/last_30d_av_hr_nbt',
              'day_nbt/last_30d_av_day_nbt', 'last_3d_amt/nbt', 'Chip Transaction', 'Online Transaction',
              'Swipe Transaction','Debit (Prepaid)',"-1","0","1","2","3","4","5","6"]]
              
       y[i] = df_split[i]["Is Fraud?"]

In [122]:
## Splitting the data into train and test data  (70:30)

## Create a STRATIFIED train-test (Hence, passing the argument 'stratify= y')
# In the first step we will split the data in training and remaining dataset
X_train = {}
y_train = {}
X_test = {}
y_test = {}
for i in range(10): 
    X_train[i], X_test[i], y_train[i], y_test[i] = train_test_split(X[i], y[i], train_size=0.7, shuffle = True, random_state = 0)

In [134]:
#building our own score
def our_score(clf, X, y):

    y_pred = clf.predict(X)
    cm = confusion_matrix(y, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    fp_rows = []

    for i in range(len(y_pred)):
        if y_pred[i] == 1 and y.values[i] == 0:
            fp_rows.append(y.index[i])

    fn_rows = []

    for i in range(len(y_pred)):
        if y_pred[i] == 0 and y.values[i] == 1:
            fn_rows.append(y.index[i])

    tp_rows = []

    for i in range(len(y_pred)):
        if y_pred[i] == 1 and y.values[i] == 1:
            tp_rows.append(y.index[i])

    tn_rows = []

    for i in range(len(y_pred)):
        if y_pred[i] == 0 and y.values[i] == 0:
            tn_rows.append(y.index[i])

    df_fp = df_cct.loc[fp_rows]
    fp_new = df_fp.loc[df_fp["abs_amount"]>=15].index

    df_tp = df_cct.loc[tp_rows]
    tp_new = df_tp.loc[df_tp["abs_amount"]>=15].index

    df_fn = df_cct.loc[fn_rows]
    fn_new = df_fn.loc[df_fn["abs_amount"]>=15].index

    our_score = -15*(len(fp_new) + len(tp_new)) + df_cct.loc[tp_new]["abs_amount"].sum() - df_cct.loc[fn_rows]["abs_amount"].sum() - df_cct.loc[tp_rows]["abs_amount"].sum()

    our_score_norm = (our_score + df_cct.loc[fn_rows]["abs_amount"].sum() + df_cct.loc[tp_rows]["abs_amount"].sum())/(df_cct.loc[tp_new]["abs_amount"].sum()+df_cct.loc[fn_new]["abs_amount"].sum() - 15*(len(tp_new)+len(fn_new)))

    return our_score_norm

In [127]:
#calculate retrieved or lost money
def retrieved_money(clf, X, y):

    y_pred = clf.predict(X)
    cm = confusion_matrix(y, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    fp_rows = []

    for i in range(len(y_pred)):
        if y_pred[i] == 1 and y.values[i] == 0:
            fp_rows.append(y.index[i])

    fn_rows = []

    for i in range(len(y_pred)):
        if y_pred[i] == 0 and y.values[i] == 1:
            fn_rows.append(y.index[i])

    tp_rows = []

    for i in range(len(y_pred)):
        if y_pred[i] == 1 and y.values[i] == 1:
            tp_rows.append(y.index[i])

    tn_rows = []

    for i in range(len(y_pred)):
        if y_pred[i] == 0 and y.values[i] == 0:
            tn_rows.append(y.index[i])

    df_fp = df_cct.loc[fp_rows]
    fp_new = df_fp.loc[df_fp["abs_amount"]>=15].index

    df_tp = df_cct.loc[tp_rows]
    tp_new = df_tp.loc[df_tp["abs_amount"]>=15].index

    df_fn = df_cct.loc[fn_rows]
    fn_new = df_fn.loc[df_fn["abs_amount"]>=15].index

    initial_loss = - df_cct.loc[fn_rows]["abs_amount"].sum() - df_cct.loc[tp_rows]["abs_amount"].sum()

    retrieved_money = -15*(len(fp_new) + len(tp_new)) + df_cct.loc[tp_new]["abs_amount"].sum() 

    max_retrieved_money = df_cct.loc[tp_new]["abs_amount"].sum()+df_cct.loc[fn_new]["abs_amount"].sum() - 15*(len(tp_new)+len(fn_new))

    return initial_loss, retrieved_money, max_retrieved_money

In [128]:
# plot confusion matrix
def plot_confusion_matrix(cm):
    """
    This function prints and plots the confusion matrix.
    """
    classes=['Non-Fraudulent','Fraudulent']
    cmap=plt.cm.Blues
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title('Confusion Matrix', fontsize=14)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes, rotation=90)
    thresh = cm.max() / 2.
    for i in range (cm.shape[0]):
        for j in range (cm.shape[1]):
            plt.text(j, i, cm[i, j],horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label', fontsize= 10)
    plt.xlabel('Predicted label', fontsize=10)

In [129]:
results_df = pd.DataFrame(columns = ["param", "our_score_cv","min_our_score_cv", 
"max_our_score_cv", "f1_cv", "precision_cv", "recall_cv"])

In [137]:
# Manual gridsearch
import itertools
from statistics import mean

grid_search_dicts =  [dict(zip(['class_weight', "n_estimators",'max_depth', 'min_samples_leaf'], [a, b, c, d]))
for a, b, c, d in itertools.product([{1:i, 0:1} for i in [10, 15, 20]], [10, 50, 100, 200, 300], [1, 5, 10, 15, 20], [1, 5, 10, 25, 50, 100, 200])]


for i in tqdm(range(len(grid_search_dicts))): #???
    my_score = []
    f1 = []
    recall = []
    precision = []
    for j in tqdm(range(10)):
        model = RandomForestClassifier(**grid_search_dicts[i])
        model.fit(X_train[j], y_train[j])
        y_pred = model.predict(X_test[j])
        my_score.append(our_score(model, X_test[j], y_test[j]))
        f1.append(f1_score(y_test[j], y_pred))
        recall.append(recall_score(y_test[j], y_pred))
        precision.append(precision_score(y_test[j], y_pred))
    results_df.loc[len(results_df)] = [grid_search_dicts[i], mean(my_score), 
    min(my_score), max(my_score), mean(f1), mean(precision), mean(recall)]

        

  0%|          | 1/525 [00:04<38:34,  4.42s/it]/opt/mamba/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  1%|          | 6/525 [00:27<39:11,  4.53s/it]/opt/mamba/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  1%|▏         | 7/525 [00:31<39:11,  4.54s/it]